<a href="https://colab.research.google.com/github/Son-mideok/e-Learning/blob/main/4%EA%B0%95_Word2Vec%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%84%EB%B2%A0%EB%94%A9_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 import

In [ ]:
import io
import re
import string
import tensorflow as tf
import tqdm

from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

## TensorBoard 로드

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

## 단어 Tokenizing

In [ ]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))
print(tokens)

8
['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']


## 단어별 id 부여 및 vocab dictionary 생성

In [ ]:
vocab, index = {}, 1 # start indexing from 1
vocab['<pad>'] = 0 # add a paddingd token, 패딩이란 글자 길이 보정해주는 특수토큰
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


## inverse dictionary 설정

In [ ]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [ ]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


## Skip-gram 모델을 위한 target, context 쌍 생성

In [ ]:
print(sentence)

The wide road shimmered in the hot sun


In [ ]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          example_sequence,
          vocabulary_size=vocab_size,
          window_size=window_size,
          negative_samples=0)
print(len(positive_skip_grams))

26


In [ ]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(6, 7): (hot, sun)
(2, 3): (wide, road)
(1, 6): (the, hot)
(1, 7): (the, sun)
(7, 1): (sun, the)


## 학습을 위한 Negative Sampling

In [ ]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
# positive example 1개당 4개의 negative example을 부여합니다.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


## Negative Example을 포함한 학습 데이터셋을 구성 

In [16]:
# Add a dimension so you can use concatenation (on the next step).
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# Concat positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label first context word as 1 (positive) followed by num_ns 0s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape target to shape (1,) and context and label to (num_ns+1,),
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [17]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 6
target_word     : hot
context_indices : [7 2 1 4 3]
context_words   : ['sun', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


In [18]:
print("target :", target)
print("contet :", context)
print("label   :", label)

target : tf.Tensor(6, shape=(), dtype=int32)
contet : tf.Tensor([7 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


## 학습용 데이터셋 형상 요약

![image.png](https://www.tensorflow.org/tutorials/text/images/word2vec_negative_sampling.png)

# 어휘(Vocabulary)별 가중치 적용하기
## 'the','is','on' 같은 단어들은 출현빈도는 높지만 새로운 정보는 많이 제공하지 못합니다. 따라서 단어별 가중치 정도를 적용해줍니다.

In [19]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):